# Mesh preprocessing

This notebook shows how to do:

1. jaw mesh segmentation
2. teeth axis calculation
3. teeth adjacency face reconstruction
4. teeth landmarks calculation
5. virtual gum generation

In [ ]:
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

def _create_colors():
    # 20 high contrast colors
    colors = [[230, 25, 75,255],[60, 180, 75,255],[255, 225, 25,255],\
            [0, 130, 200,255],[245, 130, 48,255],[145, 30, 180,255],[70, 240, 240,255],\
            [240, 50, 230,255],[210, 245, 60,255],[250, 190, 190,255],[0, 128, 128,255],\
            [230, 190, 255,255],[170, 110, 40,255],[255, 250, 200,255],[128, 0, 0,255],\
            [170, 255, 195,255],[128, 128, 0, 255]]
    #np.random.shuffle(colors)
    # gum color
    colors = [[255,255,255,255]] + colors
    return colors

def colored_mesh(mesh, label):
    COLORS = _create_colors()
    mcopy = mesh.copy()
    for i, l in enumerate(np.unique(label)):
        mcopy.visual.face_colors[np.where(label == l)[0]] = COLORS[i % 18]
    return mcopy

## Define call rules
Please modify the following code blocks based on the information you obtained from us.

In [ ]:
# Chohotech service request URL, sent with the API documentation.
base_url = "<service request URL>"

# Chohotech file service URL, sent with the API documentation.
file_server_url = "<service file server URL>"

# The authentication header must be passed in. Please keep the TOKEN confidential!!! If it is leaked, please contact us immediately to reset it. All tasks using this TOKEN will be charged to your account.
zh_token = "<your company's service Token, sent with the contact>" # All API calls must be authenticated with the token.

user_group = "APIClient" # User group, usually named APIClient.

# Your company's user_id, sent with the API documentation.
user_id = "<your company's user_id>"

# If you have received creds.json, it will be read directly below.
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # Must specify postfix, i.e., file extension
                        headers={"X-ZH-TOKEN": zh_token}) # Get signed upload URL
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # Returns a single string JSON "string", can also use json.loads(resp.text)

    resp = requests.put(upload_url, data) # No auth header is needed for uploading to the cloud storage service

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def retrieve_data(urn):
    return requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

## Mesh segmentation, axis calculation and adjacency reconstruction

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-comp-and-axis",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"ply", "data": upload_file('lower_jaw_scan.ply')},
      "jaw_type": 'Lower'
  },
  "output_config": {
      "mesh": {"type": "ply"},
      "teeth": {"type": "ply"},
      "teeth_comp": {"type": "ply"}
  }
}
result = run_job_and_get_results(json_call, 300)

In [ ]:
print('FDI list:', np.unique(result['seg_labels']))
colored_mesh(retrieve_mesh(result['mesh']), result['seg_labels']).show()

## Teeth axis calculation

If you modify the segmentation result, you can then call axis calculation API to calculate teeth axis under new segmentation. Note the input must be teeth of same jaw of same patient.

Please refer to https://www.chohotech.com/docs/cloud-en/#/module/tooth-axis-combined-1

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "tooth-axis-combined",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"meshes": result['teeth']}, # You can directly use urn here without downloading / uploading the file again!
}
result_axis = run_job_and_get_results(json_call, 300)

In [ ]:
print(result_axis['result']['result']['31'])

## Adjacency reconstruction

If you modify the segmentation result, you can then call adjacency reconstruction API to reconstruct adjacency face under new segmentation. Note the input must be teeth of same jaw of same patient.

Please refer to https://www.chohotech.com/docs/cloud-en/#/module/mesh-adj-comp-1

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "mesh-adj-comp",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"meshes": result['teeth']}, # You can directly use urn here without downloading / uploading the file again!
  "output_config": {
      "meshes": {"type": "ply"}
  }
}
result_comp = run_job_and_get_results(json_call, 300)

In [ ]:
comp_35 = retrieve_mesh(result_comp['result']['meshes']['35'])
comp_35.visual.face_colors = [255, 0, 0, 255] # show comps in red
(comp_35 + retrieve_mesh(result['teeth']['35'])).show()

## Landmarks calculations

This API returns teeth landmark positions.

Please refer to https://www.chohotech.com/docs/cloud-en/#/module/teeth-landmarks-2

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "teeth-landmarks",
  "spec_version": "2.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"meshes": result['teeth'],
                 "axis_dict": result['axis']}
}
result_landmarks = run_job_and_get_results(json_call, 300)

In [ ]:
pts = []
all_t_comps = sum([retrieve_mesh(result['teeth_comp'][k]) for k in result['teeth_comp'].keys()], None)
for m in result_landmarks['result']['result'].values():
    for k in m.values():
        for g in k:
            pts.append(g)

_, i = all_t_comps.kdtree.query(pts, k=5)
all_t_comps.visual.vertex_colors[i.ravel()] = [255, 0, 0, 255]
all_t_comps.show()

## Gum generation

This API generates a virtual gum that natually supports all input teeth.

The generated virtual gum can be displayed or used for printing model. See https://www.chohotech.com/docs/cloud-en/#/module/gum-generation-1

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "gum-generation",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"teeth_dict": result['teeth_comp']},
  "output_config": {
      "gum": {"type": "ply"}
  }
}
result_gum = run_job_and_get_results(json_call, 300)

In [ ]:
gum_mesh = retrieve_mesh(result_gum['result']['gum'])
gum_mesh.visual.face_colors = [255, 0, 0, 255] # show gum in red
(gum_mesh + sum([retrieve_mesh(result['teeth_comp'][k]) for k in result['teeth_comp'].keys()], None)).show()